## init

we'll start with
- the 6 foundations (bases)
- the 11 piles
- the one free cell

In [97]:
class Base():
    def __init__(self, suite, rank, reverse=False):
        self.suite = suite
        self.rank = rank
        self.reverse = reverse
        self.update_next()
        
    # increment / decrement
    
    def __iadd__(self, other):
        if type(other) is not int:
            return self
        self.update()
        return self
    
    def update(self):
        self.rank = self.rank + 1 if not self.reverse else self.rank - 1
        self.update_next()
        
    def update_next(self):
        self.next = self.rank + 1 if not self.reverse else self.rank - 1
    
    def __repr__(self):
        if self.suite == "t":
            if self.rank >= 0 and self.rank < 22:
                return "{:>3}".format(self.rank)
            else:
                return "---"
        else:
            if self.rank > 0:
                return "{0}{1:>2}".format(self.suite, self.rank)
            else:
                return "{0}--".format(self.suite)
    
    ## auto collection
    def collect(self, piles, cell):
        if cell and self.suite != "t":
            return piles, cell
        
        for p in piles:
            if not p:
                continue # skip empty
                
            last = p[-1]
            if last.suite != self.suite:
                continue
                
            if last.rank == self.next:
                p.pop()
                self.update()
                
        if cell.suite == self.suite and cell.rank == self.next:
            cell = None
            self.update()
        
        return piles, cell
            
hd = Base("t", -1)
tl = Base("t", 22, True)
piles = []
cell = None

SUITES = {"c": Base("c", 1), "w": Base("w", 1), "s": Base("s", 1), "p": Base("p", 1)}


In [85]:
hd += 1
tl += 1
SUITES["c"] += 1
print(hd, tl, SUITES["c"])

  0  21 c 2


now we'll define
- the card class
- some helper functions

In [76]:
class Card():
    def __init__(self, suite, rank):
        self.suite = suite
        self.rank = rank
        
    def __repr__(self):
        if self.suite == 't':
            return "{:>3}".format(self.rank)
        else:
            return "{0}{1:>2}".format(self.suite, self.rank)
        
def hash_state(p, c):
    state = ",".join(["{" + str(x) + "}" for x in range(2)]).format(p, c)
    return hash(state)

def make_deck():
    deck = []
    # major
    for rank in range(22):
        deck.append(Card("t", rank))
    # minor
    for suite in SUITES.keys():
        for rank in range(2,14):
            deck.append(Card(suite, rank))
    return deck

DECK = make_deck()
print(DECK)

[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21, c 2, c 3, c 4, c 5, c 6, c 7, c 8, c 9, c10, c11, c12, c13, w 2, w 3, w 4, w 5, w 6, w 7, w 8, w 9, w10, w11, w12, w13, s 2, s 3, s 4, s 5, s 6, s 7, s 8, s 9, s10, s11, s12, s13, p 2, p 3, p 4, p 5, p 6, p 7, p 8, p 9, p10, p11, p12, p13]


now we add pile related stuff:
- deal cards to the tableau
- pretty printing

In [130]:
import random

def new_state():
    global piles
    global cell
    global SUITES
    
    piles = []
    cell = None
    hd = Base("t", -1)
    tl = Base("t", 22, True)
    SUITES = {"c": Base("c", 1), "w": Base("w", 1), "s": Base("s", 1), "p": Base("p", 1)}

def deal():
    global piles
    random.shuffle(DECK)
    shuffled_deck = list(DECK)
    
    new_state()
    
    for i in range(11):
        piles.append([])
        if i == 5:
            continue # skip the center pile
        for j in range(7):
            card = shuffled_deck.pop()
            piles[i].append(card)
            
def show_piles(p):
    if not p:
        print("no cards have been dealt to the piles")
    height = max(len(x) for x in p)
    for i in range(height):
        row = []
        for j in range(11):
            if len(p[j]) <= i:
                row.append("   ")
            else:
                row.append(str(p[j][i]))
        print("  ".join(row))

def show_bases(c):
    ce = str(c) if c else "   "
    left = "{0}                 {1}".format(hd, tl)
    right = "[{4}] {0}  {1}  {2}  {3}".format(SUITES["c"], SUITES["w"], SUITES["s"], SUITES["p"], ce)
    print("{0}      {1}".format(left, right))

def show_board(p, c):
    show_bases(c)
    print("-0----1----2----3----4----5----6----7----8----9----X-")
    show_piles(p)        

we will then define some functions that move the cards around the board.

In [126]:
import copy

def move(src, dst):
    # -1 is the cell
    new_cell = copy.copy(cell)
    new_piles = copy.deepcopy(piles)
    
    assert src > -2 and src < 11
    assert dst > -2 and dst < 11
    assert src != dst
    
    # do cell move first
    if src == -1:
        if new_cell:
            new_piles[dst].append(cell)
        new_cell = None
    elif dst == -1:
        assert new_cell == None # it has to be empty!
        assert len(new_piles[src]) > 0
        new_cell = new_piles[src].pop()
    else:
        assert len(new_piles[src]) > 0
        new_piles[dst].append(new_piles[src].pop())
    
    return new_piles, new_cell

def get_possible_moves(piles, cell):
    movables = {}
    moves = []
    for i in range(11):
        p = piles[i]
        if len(p) > 0:
            movables[i] = p[-1]
        else:
            movables[i] = None
    movables[-1] = cell
    
    for i in movables.keys():
        for j in movables.keys():
            if i == j:
                continue
            src = movables[i]
            dst = movables[j]
            if src == None:
                continue
            elif dst == None: # if it is an empty cell
                moves.append((i, j))
            elif abs(src.rank - dst.rank) == 1 and src.suite == dst.suite:
                moves.append((i, j))
    return moves
    
def collect_all(piles, cell):
    piles, cell = hd.collect(piles, cell)
    piles, cell = tl.collect(piles, cell)
    piles, cell = SUITES["c"].collect(piles, cell)
    piles, cell = SUITES["w"].collect(piles, cell)
    piles, cell = SUITES["s"].collect(piles, cell)
    piles, cell = SUITES["p"].collect(piles, cell)
    return piles, cell

In [131]:
deal()
print(hash_state(piles, cell))
show_board(piles, cell)

piles, cell = move(6,-1)
print(hash_state(np, nc))
show_board(piles, cell)

piles, cell = move(7,6)
print(hash_state(np, nc))
show_board(piles, cell)

piles, cell = move(7,6)
print(hash_state(np, nc))
show_board(piles, cell)

5789707807324763131
---                 ---      [   ] c 1  w 1  s 1  p 1
-0----1----2----3----4----5----6----7----8----9----X-
w 9  w 6  w 4  s13    2        19  p 2  s 9  p 5  s10
  3  c10  p10    4  s 8       c 7  w13    0  c 2  p 4
p12  w 3  p 9  c12  w10       s 7    6  p 7  w12    8
c13  w 7  c 6  s 6   17       c11  p 3  c 5   10    9
 13  s 5   11    1  c 4        16  w 2   21    7  w 8
c 3  p 8  p13   12  p11         5   18  w11  s12  w 5
s11  s 2  s 4   14  s 3       p 6   20   15  c 8  c 9
5973609131606868984
---                 ---      [p 6] c 1  w 1  s 1  p 1
-0----1----2----3----4----5----6----7----8----9----X-
w 9  w 6  w 4  s13    2        19  p 2  s 9  p 5  s10
  3  c10  p10    4  s 8       c 7  w13    0  c 2  p 4
p12  w 3  p 9  c12  w10       s 7    6  p 7  w12    8
c13  w 7  c 6  s 6   17       c11  p 3  c 5   10    9
 13  s 5   11    1  c 4        16  w 2   21    7  w 8
c 3  p 8  p13   12  p11         5   18  w11  s12  w 5
s11  s 2  s 4   14  s 3             20   1

In [132]:
deal()
show_board(piles, cell)
get_possible_moves(piles, cell)

---                 ---      [   ] c 1  w 1  s 1  p 1
-0----1----2----3----4----5----6----7----8----9----X-
  7  w13    0  s 4    4       w 5   14  c13  p11    8
 21  c 2   20   11  w 6        19  w11  c12  p 8  w 2
w10  p 2    5  p 5  p 7       p 4  s 5  c 4   18  c 6
c 7  s10  p10  s12  c 3        12  s13  s 8    9  w 4
p12  p13  s11  s 6  s 9       c 9  p 3  c11    3    2
s 3    1  s 2  w 7  c10       w12  w 9  c 5   15  p 9
 13  p 6  w 3   16  s 7       w 8    6   17   10  c 8


[(0, 5),
 (0, -1),
 (1, 5),
 (1, -1),
 (2, 5),
 (2, -1),
 (3, 5),
 (3, 8),
 (3, -1),
 (4, 5),
 (4, -1),
 (6, 5),
 (6, -1),
 (7, 5),
 (7, -1),
 (8, 3),
 (8, 5),
 (8, -1),
 (9, 5),
 (9, -1),
 (10, 5),
 (10, -1)]

## search

In [7]:
visited_states = set()
